In [ ]:
import os
import numpy as np

import torchvision.transforms as transforms
from torchvision.utils import save_image

from torch.utils.data import DataLoader
from torchvision import datasets
from torch.autograd import Variable

import torch.nn as nn
import torch.nn.functional as F
import torch

os.makedirs("images", exist_ok=True)

class Args:
    n_epochs = 20
    batch_size = 64
    lr = 0.0002
    b1 = 0.5
    b2 = 0.999
    n_cpu = 8
    latent_dim = 100
    img_size = 32
    channels = 1
    sample_interval = 400

opt = Args()

cuda = True if torch.cuda.is_available() else False


def weights_init_normal(m):
    classname = m.__class__.__name__
    if classname.find("Conv") != -1:
        torch.nn.init.normal_(m.weight.data, 0.0, 0.02)
    elif classname.find("BatchNorm2d") != -1:
        torch.nn.init.normal_(m.weight.data, 1.0, 0.02)
        torch.nn.init.constant_(m.bias.data, 0.0)


class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()

        self.init_size = opt.img_size // 4
        self.l1 = nn.Sequential(nn.Linear(opt.latent_dim, 128 * self.init_size ** 2))

        self.conv_blocks = nn.Sequential(
            nn.BatchNorm2d(128),
            nn.Upsample(scale_factor=2),
            nn.Conv2d(128, 128, 3, stride=1, padding=1),
            nn.BatchNorm2d(128, 0.8),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Upsample(scale_factor=2),
            nn.Conv2d(128, 64, 3, stride=1, padding=1),
            nn.BatchNorm2d(64, 0.8),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(64, opt.channels, 3, stride=1, padding=1),
            nn.Tanh(),
        )

    def forward(self, z):
        out = self.l1(z)
        out = out.view(out.shape[0], 128, self.init_size, self.init_size)
        img = self.conv_blocks(out)
        return img


class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()

        def discriminator_block(in_filters, out_filters, bn=True):
            block = [nn.Conv2d(in_filters, out_filters, 3, 2, 1), nn.LeakyReLU(0.2, inplace=True), nn.Dropout2d(0.25)]
            if bn:
                block.append(nn.BatchNorm2d(out_filters, 0.8))
            return block

        self.model = nn.Sequential(
            *discriminator_block(opt.channels, 16, bn=False),
            *discriminator_block(16, 32),
            *discriminator_block(32, 64),
            *discriminator_block(64, 128),
        )

        # The height and width of downsampled image
        ds_size = opt.img_size // 2 ** 4
        self.adv_layer = nn.Sequential(nn.Linear(128 * ds_size ** 2, 1), nn.Sigmoid())

    def forward(self, img):
        out = self.model(img)
        out = out.view(out.shape[0], -1)
        validity = self.adv_layer(out)

        return validity


# Loss function
adversarial_loss = torch.nn.BCELoss()

# Initialize generator and discriminator
generator = Generator()
discriminator = Discriminator()

if cuda:
    generator.cuda()
    discriminator.cuda()
    adversarial_loss.cuda()


# Initialize weights
generator.apply(weights_init_normal)
discriminator.apply(weights_init_normal)

# Configure data loader
os.makedirs("./data/mnist", exist_ok=True)
dataloader = torch.utils.data.DataLoader(
    datasets.MNIST(
        "./data/mnist",
        train=True,
        download=True,
        transform=transforms.Compose(
            [transforms.Resize(opt.img_size), transforms.ToTensor(), transforms.Normalize([0.5], [0.5])]
        ),
    ),
    batch_size=opt.batch_size,
    shuffle=True,
)

# Optimizers
optimizer_G = torch.optim.Adam(generator.parameters(), lr=opt.lr, betas=(opt.b1, opt.b2))
optimizer_D = torch.optim.Adam(discriminator.parameters(), lr=opt.lr, betas=(opt.b1, opt.b2))

Tensor = torch.cuda.FloatTensor if cuda else torch.FloatTensor

# ----------
#  Training
# ----------

for epoch in range(opt.n_epochs):
    for i, (imgs, _) in enumerate(dataloader):

        # Adversarial ground truths
        valid = Variable(Tensor(imgs.shape[0], 1).fill_(1.0), requires_grad=False)
        fake = Variable(Tensor(imgs.shape[0], 1).fill_(0.0), requires_grad=False)

        # Configure input
        real_imgs = Variable(imgs.type(Tensor))

        # -----------------
        #  Train Generator
        # -----------------

        optimizer_G.zero_grad()

        # Sample noise as generator input
        z = Variable(Tensor(np.random.normal(0, 1, (imgs.shape[0], opt.latent_dim))))

        # Generate a batch of images
        gen_imgs = generator(z)

        # Loss measures generator's ability to fool the discriminator
        g_loss = adversarial_loss(discriminator(gen_imgs), valid)

        g_loss.backward()
        optimizer_G.step()

        # ---------------------
        #  Train Discriminator
        # ---------------------

        optimizer_D.zero_grad()

        # Measure discriminator's ability to classify real from generated samples
        real_loss = adversarial_loss(discriminator(real_imgs), valid)
        fake_loss = adversarial_loss(discriminator(gen_imgs.detach()), fake)
        d_loss = (real_loss + fake_loss) / 2

        d_loss.backward()
        optimizer_D.step()

        print(
            "[Epoch %d/%d] [Batch %d/%d] [D loss: %f] [G loss: %f]"
            % (epoch, opt.n_epochs, i, len(dataloader), d_loss.item(), g_loss.item())
        )

        batches_done = epoch * len(dataloader) + i
        if batches_done % opt.sample_interval == 0:
            save_image(gen_imgs.data[:25], "images/%d.png" % batches_done, nrow=5, normalize=True)

# 在训练循环之后，添加模型保存的逻辑
if epoch == opt.n_epochs - 1:  # 在最后一个epoch结束时保存模型
    torch.save(generator.state_dict(), "generator_weights.pth")  # 保存生成器权重
    torch.save(discriminator.state_dict(), "discriminator_weights.pth")  # 可选：保存判别器权重

# 在训练结束后，使用保存的生成器模型生成200张图像
# 首先确保在适当的位置加载生成器的权重
generator.load_state_dict(torch.load("generator_weights.pth"))
generator.eval()  # 设置为评估模式，关闭dropout等训练时的行为

流式输出内容被截断，只能显示最后 5000 行内容。
[Epoch 14/20] [Batch 628/938] [D loss: 0.610923] [G loss: 0.902336]
[Epoch 14/20] [Batch 629/938] [D loss: 0.666029] [G loss: 1.057574]
[Epoch 14/20] [Batch 630/938] [D loss: 0.507800] [G loss: 0.543915]
[Epoch 14/20] [Batch 631/938] [D loss: 0.628291] [G loss: 0.816878]
[Epoch 14/20] [Batch 632/938] [D loss: 0.550982] [G loss: 0.848421]
[Epoch 14/20] [Batch 633/938] [D loss: 0.677921] [G loss: 0.832208]
[Epoch 14/20] [Batch 634/938] [D loss: 0.579918] [G loss: 1.086726]
[Epoch 14/20] [Batch 635/938] [D loss: 0.644832] [G loss: 0.954838]
[Epoch 14/20] [Batch 636/938] [D loss: 0.655515] [G loss: 0.815497]
[Epoch 14/20] [Batch 637/938] [D loss: 0.667931] [G loss: 0.927454]
[Epoch 14/20] [Batch 638/938] [D loss: 0.612136] [G loss: 0.913995]
[Epoch 14/20] [Batch 639/938] [D loss: 0.629412] [G loss: 0.960222]
[Epoch 14/20] [Batch 640/938] [D loss: 0.555465] [G loss: 0.930995]
[Epoch 14/20] [Batch 641/938] [D loss: 0.695843] [G loss: 0.688191]
[Epoch 14/20] [Batch 

Generator(
  (l1): Sequential(
    (0): Linear(in_features=100, out_features=8192, bias=True)
  )
  (conv_blocks): Sequential(
    (0): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (1): Upsample(scale_factor=2.0, mode='nearest')
    (2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): BatchNorm2d(128, eps=0.8, momentum=0.1, affine=True, track_running_stats=True)
    (4): LeakyReLU(negative_slope=0.2, inplace=True)
    (5): Upsample(scale_factor=2.0, mode='nearest')
    (6): Conv2d(128, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): BatchNorm2d(64, eps=0.8, momentum=0.1, affine=True, track_running_stats=True)
    (8): LeakyReLU(negative_slope=0.2, inplace=True)
    (9): Conv2d(64, 1, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (10): Tanh()
  )
)

In [ ]:
# 确保在生成图像时，gen_img的形状正确
# 假设generator的输出应该是 [batch_size, channels, height, width]
# 而我们只需要生成单个图像，所以理想情况下，gen_img的形状应该是 [1, channels, height, width]

with torch.no_grad():
    for i in range(500):
        z = Tensor(np.random.normal(0, 1, (1, opt.latent_dim)))  # 假定opt.latent_dim是生成器所需噪声向量的长度
        gen_img = generator(z)  # 注意这里不加.unsqueeze(0)，因为我们每次只想生成一个样本

        # 确保gen_img的形状适合保存为单个图像，例如 [1, 1, 32, 32] 或者 [1, 3, 32, 32] 对于RGB图像
        if gen_img.shape != (1, 1, 32, 32):
            print("警告：生成图像的形状不符合预期，请检查生成器输出。")
        else:
            img_path = f"generated_images/image_{i:04d}.png"
            save_image(gen_img, img_path, normalize=True, nrow=1)  # nrow=1 表示每行一个图像，避免重叠

In [ ]:
!zip -r generated_images.zip generated_images

updating: generated_images/ (stored 0%)
  adding: generated_images/image_0191.png (stored 0%)
  adding: generated_images/epoch5_image_0141.png (stored 0%)
  adding: generated_images/image_0440.png (stored 0%)
  adding: generated_images/epoch5_image_0008.png (stored 0%)
  adding: generated_images/image_0037.png (stored 0%)
  adding: generated_images/image_0090.png (stored 0%)
  adding: generated_images/image_0363.png (stored 0%)
  adding: generated_images/image_0270.png (stored 0%)
  adding: generated_images/epoch5_image_0003.png (stored 0%)
  adding: generated_images/image_0459.png (stored 0%)
  adding: generated_images/epoch5_image_0009.png (stored 0%)
  adding: generated_images/epoch5_image_0121.png (stored 0%)
  adding: generated_images/image_0215.png (stored 0%)
  adding: generated_images/image_0423.png (stored 0%)
  adding: generated_images/image_0212.png (stored 0%)
  adding: generated_images/epoch5_image_0069.png (stored 0%)
  adding: generated_images/epoch5_image_0110.png (stor